In [17]:
#imports
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [18]:
#training data
df_train=pd.read_csv('/content/drive/My Drive/minor/english_dataset/eng23train.csv', encoding="latin-1")
df_train.head()

,text_id,text,task_1,task_2,task_3
0,hasoc_en_2,@politico no we should remember very clearly t...,0,1,1
1,hasoc_en_8,ados trendingnow blacklivesmatter justice fuck...,0,0,1
2,hasoc_en_12,i dont know how much more i can take ## is a c...,0,1,1
3,hasoc_en_16,good work @icc keep going just destroy the who...,0,0,1
4,hasoc_en_24,shameonicc 1 icc on dhonis gloves ...,0,1,1


In [19]:
#training and validation split
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_train['text'], df_train['task_2'],test_size=0.20,random_state=140) 

In [20]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder() #label encoder
train_y = encoder.fit_transform(train_y) #encoding label for training data
valid_y = encoder.fit_transform(valid_y) #encoding label for validation data

In [21]:
#testing data
df_test=pd.read_csv('/content/drive/My Drive/minor/english_dataset/eng23test.csv', encoding="latin-1") 
test_y = encoder.fit_transform(df_test['task_2']) #encoding the label of testing data

In [22]:
# create a tfidf vectorizer object 
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=25999)
tfidf_vect.fit(df_train['text'])

# transform the training and validation data using count vectorizer object
xtrain_tfidf =  tfidf_vect.transform(train_x)  #get tfidf vector features for training data
xvalid_tfidf =  tfidf_vect.transform(valid_x)  #get tfidf vector features for validation data
x_tfidf =  tfidf_vect.transform(df_test['text'])  #get tfidf vector features for testing data

In [23]:
def model(classifier, feature_vector_train, label, feature_vector_valid,feature_vector_test):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions_valid = classifier.predict(feature_vector_valid)

    # predict the labels on testing dataset
    predictions_test = classifier.predict(feature_vector_test)

    print("classification report for validation")
    print(metrics.classification_report(predictions_valid,valid_y))
    print("classification report for testing")
    print(metrics.classification_report(predictions_test,test_y))


In [24]:
# calling the xgboost classifier model for training ,validation and testing
model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc(),x_tfidf.tocsc())

classification report for validation
              precision    recall  f1-score   support

           0       0.74      0.71      0.73       135
           1       0.89      0.71      0.79       296
           2       0.12      0.50      0.20        22

    accuracy                           0.70       453
   macro avg       0.58      0.64      0.57       453
weighted avg       0.81      0.70      0.74       453

classification report for testing
              precision    recall  f1-score   support

           0       0.74      0.68      0.71       101
           1       0.93      0.62      0.74       185
           2       0.01      0.50      0.03         2

    accuracy                           0.64       288
   macro avg       0.56      0.60      0.49       288
weighted avg       0.86      0.64      0.73       288

